In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import numpy as np
import wandb
from sklearn.preprocessing import MinMaxScaler
import plotly.graph_objects as go

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Hesaplamalar için kullanılacak cihaz: {device}')

wandb.login()

Hesaplamalar için kullanılacak cihaz: cuda


wandb: Currently logged in as: melihak635442 (melihak635442-bites) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [3]:
# Sıralı veri oluşturma fonksiyonu
def create_sequences(data, lookback_window):
    X, y = [], []
    for i in range(len(data) - lookback_window):
        feature = data[i:(i + lookback_window)]
        target = data[i + lookback_window]
        X.append(feature)
        y.append(target)
    return torch.FloatTensor(np.array(X)), torch.FloatTensor(np.array(y))

In [4]:
# Veriyi yükle ve ön işle
df = pd.read_csv('genel_sunucu_yuku_tum_metrikler.csv', index_col='timestamp')
df.index = pd.to_datetime(df.index, unit='s')
ts_data = df[['avg_min_cpu', 'avg_max_cpu', 'avg_avg_cpu']].values
scaler = MinMaxScaler(feature_range=(0, 1))
ts_data_scaled = scaler.fit_transform(ts_data)
train_size = int(len(ts_data_scaled) * 0.8)
train_data, test_data = ts_data_scaled[:train_size], ts_data_scaled[train_size:]

lookback = 12

# PyTorch Dataset ve DataLoader'ları oluştur
X_train, y_train = create_sequences(train_data, lookback)
X_test, y_test = create_sequences(test_data, lookback)

train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

print("Veri setleri ve DataLoader'lar hazır.")


Veri setleri ve DataLoader'lar hazır.


In [5]:
class LSTMModel(nn.Module):
    def __init__(self, input_size=3, hidden_layer_size=50, num_layers=1, dropout=0.0, output_size=3):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_layer_size,
            num_layers=num_layers,
            dropout=dropout,
            batch_first=True
        )
        self.linear = nn.Linear(hidden_layer_size, output_size)

    def forward(self, input_seq):
        lstm_out, _ = self.lstm(input_seq)
        predictions = self.linear(lstm_out[:, -1, :])
        return predictions

In [6]:
def train_and_evaluate_lstm(config):
    #  WandB Deneyini Başlat
    run = wandb.init(project="bites-cpu-forecasting",
                      config=config, 
                      reinit=True,
                      name=f"LSTM_h{config['hidden_dim']}_l{config['num_layers']}")
    
    #  DataLoader'ları Oluştur
    train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=config['batch_size'])
    
    #  Modeli Oluştur
    model = LSTMModel(
        hidden_layer_size=config['hidden_dim'],
        num_layers=config['num_layers'],
        dropout=config['dropout']
    ).to(device)
    
    #  Optimizer ve Loss Fonksiyonunu Tanımla
    optimizer = optim.Adam(model.parameters(), lr=config['lr'])
    criterion = nn.MSELoss()
    
    #  Modeli ve Gradyanları İzle
    wandb.watch(model, criterion, log="all", log_freq=10)

    #  Eğitim Döngüsü
    print(f"\n--- Deney Başladı: {run.name} ---")
    for epoch in range(config['epochs']):
        # --- Eğitim Adımı ---
        model.train()
        total_train_loss = 0
        for batch_x, batch_y in train_loader:
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)
            optimizer.zero_grad()
            outputs = model(batch_x)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()
            total_train_loss += loss.item()
        
        # Ortalama eğitim loss'unu hesapla
        avg_train_loss = total_train_loss / len(train_loader)

        # --- Değerlendirme Adımı ---
        model.eval()
        total_test_loss = 0
        with torch.no_grad():
            for batch_x, batch_y in test_loader:
                batch_x, batch_y = batch_x.to(device), batch_y.to(device)
                outputs = model(batch_x)
                total_test_loss += criterion(outputs, batch_y).item()
        
        # Ortalama test loss'unu hesapla
        avg_test_loss = total_test_loss / len(test_loader)
        
        # 7. Her Epoch Sonunda WandB'ye Logla
        wandb.log({
            "epoch": epoch, 
            "train_loss": avg_train_loss,
            "test_loss": avg_test_loss
        })
        print(f"Epoch {epoch+1}/{config['epochs']} | Train Loss: {avg_train_loss:.6f} | Test Loss: {avg_test_loss:.6f}")

    # 8. Modeli Kaydet ve Deneyi Bitir
    model_path = f"model_path/lstm_{run.name}.pt"
    torch.save(model.state_dict(), model_path)
    wandb.save(model_path)
    run.finish()


In [7]:
# Denenecek farklı ve daha çeşitli hiperparametre setleri
configs = [
    # 1. Küçük ve Hızlı Model
    {'hidden_dim': 50, 'num_layers': 1, 'dropout': 0.1, 'lr': 1e-3},
    
    # 2. Orta Boyutlu Model
    {'hidden_dim': 100, 'num_layers': 2, 'dropout': 0.2, 'lr': 5e-4},
    
    # 3. Daha Derin Model
    {'hidden_dim': 128, 'num_layers': 3, 'dropout': 0.3, 'lr': 1e-4},
    
    # 4. Daha Geniş ama Sığ Model
    {'hidden_dim': 256, 'num_layers': 1, 'dropout': 0.2, 'lr': 5e-4},
    
    # 5. Farklı bir kombinasyon
    {'hidden_dim': 128, 'num_layers': 2, 'dropout': 0.1, 'lr': 8e-4}
]

# Tüm deneyler için ortak olan sabit parametreler
base_config = {
    'batch_size': 64,
    'epochs': 20
}

# WandB'ye giriş yaptığınızdan emin olun: terminalde 'wandb login'

# Tüm konfigürasyonları sırayla dene
for cfg in configs:
    print(f"\n--- Yeni LSTM Deneyi Başlatılıyor ---")
    print(f"Parametreler: {cfg}")
    
    # Sabit ve denenecek parametreleri birleştir
    full_config = {**base_config, **cfg}
    
    # Ana eğitim fonksiyonunu bu konfigürasyon ile çağır
    train_and_evaluate_lstm(full_config)
    
    # Deneyler arası GPU belleğini temizle
    torch.cuda.empty_cache()



--- Yeni LSTM Deneyi Başlatılıyor ---
Parametreler: {'hidden_dim': 50, 'num_layers': 1, 'dropout': 0.1, 'lr': 0.001}


wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.


/home/melih/Documents/Bites_CPU_Analizi/bites_cpu_venv/lib/python3.13/site-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn(



--- Deney Başladı: LSTM_h50_l1 ---
Epoch 1/20 | Train Loss: 0.017243 | Test Loss: 0.004881
Epoch 2/20 | Train Loss: 0.003234 | Test Loss: 0.002613
Epoch 3/20 | Train Loss: 0.002484 | Test Loss: 0.002528
Epoch 4/20 | Train Loss: 0.002330 | Test Loss: 0.002297
Epoch 5/20 | Train Loss: 0.002227 | Test Loss: 0.002245
Epoch 6/20 | Train Loss: 0.002203 | Test Loss: 0.002249
Epoch 7/20 | Train Loss: 0.002110 | Test Loss: 0.002124
Epoch 8/20 | Train Loss: 0.002118 | Test Loss: 0.002215
Epoch 9/20 | Train Loss: 0.002049 | Test Loss: 0.002057
Epoch 10/20 | Train Loss: 0.002027 | Test Loss: 0.002216
Epoch 11/20 | Train Loss: 0.002032 | Test Loss: 0.002050
Epoch 12/20 | Train Loss: 0.002018 | Test Loss: 0.002160
Epoch 13/20 | Train Loss: 0.002000 | Test Loss: 0.002355
Epoch 14/20 | Train Loss: 0.001991 | Test Loss: 0.002163
Epoch 15/20 | Train Loss: 0.002004 | Test Loss: 0.002060
Epoch 16/20 | Train Loss: 0.001990 | Test Loss: 0.001993
Epoch 17/20 | Train Loss: 0.001980 | Test Loss: 0.002258
Epoc

epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
test_loss,█▃▂▂▂▂▁▂▁▂▁▁▂▁▁▁▂▁▁▁
train_loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,19
test_loss,0.00205
train_loss,0.00196


...


--- Deney Başladı: LSTM_h256_l1 ---
Epoch 1/20 | Train Loss: 0.017602 | Test Loss: 0.003995
Epoch 2/20 | Train Loss: 0.003720 | Test Loss: 0.002936
Epoch 3/20 | Train Loss: 0.002719 | Test Loss: 0.002630
Epoch 4/20 | Train Loss: 0.002570 | Test Loss: 0.002503
Epoch 5/20 | Train Loss: 0.002420 | Test Loss: 0.002319
Epoch 6/20 | Train Loss: 0.002326 | Test Loss: 0.002663
Epoch 7/20 | Train Loss: 0.002264 | Test Loss: 0.002384
Epoch 8/20 | Train Loss: 0.002237 | Test Loss: 0.002237
Epoch 9/20 | Train Loss: 0.002148 | Test Loss: 0.002245
Epoch 10/20 | Train Loss: 0.002122 | Test Loss: 0.002392
Epoch 11/20 | Train Loss: 0.002083 | Test Loss: 0.002458
Epoch 12/20 | Train Loss: 0.002138 | Test Loss: 0.002208
Epoch 13/20 | Train Loss: 0.002089 | Test Loss: 0.002234
Epoch 14/20 | Train Loss: 0.002039 | Test Loss: 0.002092
Epoch 15/20 | Train Loss: 0.002026 | Test Loss: 0.002056
Epoch 16/20 | Train Loss: 0.002016 | Test Loss: 0.002280
Epoch 17/20 | Train Loss: 0.002051 | Test Loss: 0.002136
Epo

epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
test_loss,█▄▃▃▂▃▂▂▂▂▃▂▂▁▁▂▁▁▁▁
train_loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,19
test_loss,0.00209
train_loss,0.00201



--- Yeni LSTM Deneyi Başlatılıyor ---
Parametreler: {'hidden_dim': 128, 'num_layers': 2, 'dropout': 0.1, 'lr': 0.0008}



--- Deney Başladı: LSTM_h128_l2 ---
Epoch 1/20 | Train Loss: 0.016138 | Test Loss: 0.003918
Epoch 2/20 | Train Loss: 0.003654 | Test Loss: 0.003152
Epoch 3/20 | Train Loss: 0.002700 | Test Loss: 0.002611
Epoch 4/20 | Train Loss: 0.002533 | Test Loss: 0.002721
Epoch 5/20 | Train Loss: 0.002389 | Test Loss: 0.002520
Epoch 6/20 | Train Loss: 0.002333 | Test Loss: 0.002349
Epoch 7/20 | Train Loss: 0.002219 | Test Loss: 0.002167
Epoch 8/20 | Train Loss: 0.002192 | Test Loss: 0.002151
Epoch 9/20 | Train Loss: 0.002156 | Test Loss: 0.002163
Epoch 10/20 | Train Loss: 0.002135 | Test Loss: 0.002117
Epoch 11/20 | Train Loss: 0.002170 | Test Loss: 0.002119
Epoch 12/20 | Train Loss: 0.002114 | Test Loss: 0.002073
Epoch 13/20 | Train Loss: 0.002164 | Test Loss: 0.002096
Epoch 14/20 | Train Loss: 0.002060 | Test Loss: 0.002121
Epoch 15/20 | Train Loss: 0.002086 | Test Loss: 0.002056
Epoch 16/20 | Train Loss: 0.002113 | Test Loss: 0.002350
Epoch 17/20 | Train Loss: 0.002077 | Test Loss: 0.002203
Epo

epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
test_loss,█▅▃▃▃▂▁▁▁▁▁▁▁▁▁▂▂▂▁▁
train_loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,19
test_loss,0.00215
train_loss,0.00204


In [8]:
# Denenecek farklı hiperparametre setleri
# Her bir sözlük, ayrı bir WandB deneyi olarak çalıştırılacak
configs = [
    {'hidden_dim': 50, 'num_layers': 1, 'dropout': 0.1, 'lr': 1e-3},
    {'hidden_dim': 100, 'num_layers': 2, 'dropout': 0.2, 'lr': 5e-4},
    {'hidden_dim': 128, 'num_layers': 2, 'dropout': 0.3, 'lr': 1e-4} # Üçüncü bir deneme
]

# Tüm deneyler için ortak olan sabit parametreler
base_config = {
    'batch_size': 64,
    'epochs': 20
}

# WandB'ye giriş yaptığınızdan emin olun: terminalde 'wandb login'

# Tüm konfigürasyonları sırayla dene
for cfg in configs:
    print(f"\n--- Yeni Deney Başlatılıyor ---")
    print(f"Parametreler: {cfg}")
    
    # Sabit ve denenecek parametreleri birleştir
    full_config = {**base_config, **cfg}
    
    # Ana eğitim fonksiyonunu bu konfigürasyon ile çağır
    train_and_evaluate_lstm(full_config)
    
    # Deneyler arası GPU belleğini temizleyerek olası hataları önle
    torch.cuda.empty_cache()


--- Yeni Deney Başlatılıyor ---
Parametreler: {'hidden_dim': 50, 'num_layers': 1, 'dropout': 0.1, 'lr': 0.001}


/home/melih/Documents/Bites_CPU_Analizi/bites_cpu_venv/lib/python3.13/site-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn(



--- Deney Başladı: LSTM_h50_l1 ---
Epoch 1/20 | Train Loss: 0.024056 | Test Loss: 0.005175
Epoch 2/20 | Train Loss: 0.004291 | Test Loss: 0.003848
Epoch 3/20 | Train Loss: 0.003498 | Test Loss: 0.003341
Epoch 4/20 | Train Loss: 0.002611 | Test Loss: 0.002485
Epoch 5/20 | Train Loss: 0.002388 | Test Loss: 0.002635
Epoch 6/20 | Train Loss: 0.002315 | Test Loss: 0.002596
Epoch 7/20 | Train Loss: 0.002256 | Test Loss: 0.002388
Epoch 8/20 | Train Loss: 0.002260 | Test Loss: 0.002200
Epoch 9/20 | Train Loss: 0.002137 | Test Loss: 0.002315
Epoch 10/20 | Train Loss: 0.002161 | Test Loss: 0.002227
Epoch 11/20 | Train Loss: 0.002108 | Test Loss: 0.002172
Epoch 12/20 | Train Loss: 0.002147 | Test Loss: 0.002193
Epoch 13/20 | Train Loss: 0.002066 | Test Loss: 0.002128
Epoch 14/20 | Train Loss: 0.002063 | Test Loss: 0.002077
Epoch 15/20 | Train Loss: 0.002058 | Test Loss: 0.002252
Epoch 16/20 | Train Loss: 0.002050 | Test Loss: 0.002213
Epoch 17/20 | Train Loss: 0.002031 | Test Loss: 0.002046
Epoc

epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
test_loss,█▅▄▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁
train_loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,19
test_loss,0.00212
train_loss,0.002



--- Yeni Deney Başlatılıyor ---
Parametreler: {'hidden_dim': 100, 'num_layers': 2, 'dropout': 0.2, 'lr': 0.0005}



--- Deney Başladı: LSTM_h100_l2 ---
Epoch 1/20 | Train Loss: 0.015472 | Test Loss: 0.005495
Epoch 2/20 | Train Loss: 0.004514 | Test Loss: 0.003631
Epoch 3/20 | Train Loss: 0.003004 | Test Loss: 0.002714
Epoch 4/20 | Train Loss: 0.002768 | Test Loss: 0.002942
Epoch 5/20 | Train Loss: 0.002619 | Test Loss: 0.002506
Epoch 6/20 | Train Loss: 0.002526 | Test Loss: 0.002733
Epoch 7/20 | Train Loss: 0.002446 | Test Loss: 0.002497
Epoch 8/20 | Train Loss: 0.002364 | Test Loss: 0.002347
Epoch 9/20 | Train Loss: 0.002316 | Test Loss: 0.002528
Epoch 10/20 | Train Loss: 0.002326 | Test Loss: 0.002389
Epoch 11/20 | Train Loss: 0.002257 | Test Loss: 0.002192
Epoch 12/20 | Train Loss: 0.002219 | Test Loss: 0.002136
Epoch 13/20 | Train Loss: 0.002182 | Test Loss: 0.002317
Epoch 14/20 | Train Loss: 0.002185 | Test Loss: 0.002151
Epoch 15/20 | Train Loss: 0.002159 | Test Loss: 0.002090
Epoch 16/20 | Train Loss: 0.002153 | Test Loss: 0.002190
Epoch 17/20 | Train Loss: 0.002194 | Test Loss: 0.002270
Epo

epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
test_loss,█▄▂▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▂
train_loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,19
test_loss,0.0024
train_loss,0.00209



--- Yeni Deney Başlatılıyor ---
Parametreler: {'hidden_dim': 128, 'num_layers': 2, 'dropout': 0.3, 'lr': 0.0001}



--- Deney Başladı: LSTM_h128_l2 ---
Epoch 1/20 | Train Loss: 0.049635 | Test Loss: 0.010368
Epoch 2/20 | Train Loss: 0.007692 | Test Loss: 0.004566
Epoch 3/20 | Train Loss: 0.005121 | Test Loss: 0.004915
Epoch 4/20 | Train Loss: 0.004929 | Test Loss: 0.004945
Epoch 5/20 | Train Loss: 0.004735 | Test Loss: 0.005095
Epoch 6/20 | Train Loss: 0.004508 | Test Loss: 0.004550
Epoch 7/20 | Train Loss: 0.003959 | Test Loss: 0.003172
Epoch 8/20 | Train Loss: 0.003414 | Test Loss: 0.003079
Epoch 9/20 | Train Loss: 0.003301 | Test Loss: 0.003152
Epoch 10/20 | Train Loss: 0.003161 | Test Loss: 0.003066
Epoch 11/20 | Train Loss: 0.003071 | Test Loss: 0.002914
Epoch 12/20 | Train Loss: 0.003000 | Test Loss: 0.002838
Epoch 13/20 | Train Loss: 0.002974 | Test Loss: 0.002778
Epoch 14/20 | Train Loss: 0.002859 | Test Loss: 0.002728
Epoch 15/20 | Train Loss: 0.002826 | Test Loss: 0.003007
Epoch 16/20 | Train Loss: 0.002769 | Test Loss: 0.002681
Epoch 17/20 | Train Loss: 0.002695 | Test Loss: 0.002711
Epo

epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
test_loss,█▃▃▃▃▃▂▁▂▁▁▁▁▁▁▁▁▁▁▁
train_loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,19
test_loss,0.00253
train_loss,0.0026


In [10]:
import wandb
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import plotly.graph_objects as go

def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / (y_true + 1e-8))) * 100

# --- 1. Adım: En İyi Deneyin Hiperparametrelerini ve Model Dosyasını Belirle ---

# WandB arayüzünden 'mild-planet-32' deneyinin "Config" sekmesinden
# bu bilgileri almalısın. 
best_run_config = {
    'hidden_dim': 50,
    'num_layers': 1,
    'dropout': 0.1,
    'lr': 0.001,
    'batch_size': 64,
    'epochs':20
}
best_model_filename = "model_path/lstm_LSTM_h50_l1.pt" 

# --- 2. Adım: En İyi Modelin Ağırlıklarını Yükle ---

# Önce en iyi hiperparametrelerle modelin boş bir kopyasını oluştur
final_model = LSTMModel(
    hidden_layer_size=best_run_config['hidden_dim'],
    num_layers=best_run_config['num_layers'],
    dropout=best_run_config['dropout']
).to(device)

# Kayıtlı ağırlıkları modele yükle
try:
    final_model.load_state_dict(torch.load(best_model_filename))
    print(f"'{best_model_filename}' dosyasından ağırlıklar başarıyla yüklendi.")
except FileNotFoundError:
    print(f"HATA: '{best_model_filename}' dosyası bulunamadı.")
    # Gerekirse bu hücreyi durdurabilirsin
    # assert False, "Model dosyası bulunamadı, devam edilemiyor."

# --- 3. Adım: Yüklenen Model ile Tahmin Yap ---

final_model.eval()
test_predictions = []
with torch.no_grad():
    for seq, label in zip(X_test, y_test):
        seq_gpu = seq.unsqueeze(0).to(device)
        y_pred = final_model(seq_gpu)
        test_predictions.append(y_pred.squeeze(0).cpu().numpy())

predictions_original_scale = scaler.inverse_transform(np.array(test_predictions))
y_test_original_scale = scaler.inverse_transform(y_test.numpy())

# --- 4. Adım: Nihai Metrikleri ve Grafiği Hazırla ---

target_columns = ['avg_min_cpu', 'avg_max_cpu', 'avg_avg_cpu']
final_metrics = {}

print("\n--- Nihai Modelin Performans Metrikleri ---")
for i, column in enumerate(target_columns):
    y_true_col = y_test_original_scale[:, i]
    y_pred_col = predictions_original_scale[:, i]
    
    mae = mean_absolute_error(y_true_col, y_pred_col)
    rmse = np.sqrt(mean_squared_error(y_true_col, y_pred_col))
    r2 = r2_score(y_true_col, y_pred_col)
    mape = mean_absolute_percentage_error(y_true_col, y_pred_col)
    
    final_metrics[column] = {'MAE': mae, 'RMSE': rmse, 'R2': r2, 'MAPE': mape}
    
    print(f"\n--- Metrik: '{column}' ---")
    print(f"  MAE: {mae:.2f}, RMSE: {rmse:.2f}, R²: {r2:.2f}, MAPE: {mape:.2f}%")

# Nihai grafiği oluştur
final_fig = go.Figure()
for i, column in enumerate(target_columns):
    final_fig.add_trace(go.Scatter(x=df.index[train_size+lookback:], y=y_test_original_scale[:, i], mode='lines', name=f'Gerçek {column}'))
    final_fig.add_trace(go.Scatter(x=df.index[train_size+lookback:], y=predictions_original_scale[:, i], mode='lines', name=f'Tahmin {column}', line=dict(dash='dash')))
final_fig.update_layout(title='En İyi LSTM Modelinin Nihai Tahmin Performansı', xaxis_title='Zaman', yaxis_title='Ortalama CPU Yükü (%)')
final_fig.show()

# --- 5. Adım: Nihai Sonuçları WandB'ye Logla ---

print("\nNihai sonuçlar WandB'ye loglanıyor...")
run = wandb.init(project="bites-cpu-forecasting", name="Final_Evaluation_Best_LSTM", config=best_run_config)

# Loglanacak metrikleri düzenle
log_dict = {}
for col_name, metrics in final_metrics.items():
    for metric_name, value in metrics.items():
        log_dict[f"{col_name}_{metric_name}"] = value

# Metrikleri ve son grafiği logla
log_dict["Final Tahmin Grafiği"] = final_fig
wandb.log(log_dict)

run.finish()
print("Loglama tamamlandı.")

/home/melih/Documents/Bites_CPU_Analizi/bites_cpu_venv/lib/python3.13/site-packages/torch/nn/modules/rnn.py:123: UserWarning:

dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1



'model_path/lstm_LSTM_h50_l1.pt' dosyasından ağırlıklar başarıyla yüklendi.

--- Nihai Modelin Performans Metrikleri ---

--- Metrik: 'avg_min_cpu' ---
  MAE: 0.06, RMSE: 0.09, R²: 0.85, MAPE: 1.02%

--- Metrik: 'avg_max_cpu' ---
  MAE: 0.21, RMSE: 0.30, R²: 0.73, MAPE: 1.04%

--- Metrik: 'avg_avg_cpu' ---
  MAE: 0.11, RMSE: 0.14, R²: 0.82, MAPE: 1.11%



Nihai sonuçlar WandB'ye loglanıyor...


avg_avg_cpu_MAE,▁
avg_avg_cpu_MAPE,▁
avg_avg_cpu_R2,▁
avg_avg_cpu_RMSE,▁
avg_max_cpu_MAE,▁
avg_max_cpu_MAPE,▁
avg_max_cpu_R2,▁
avg_max_cpu_RMSE,▁
avg_min_cpu_MAE,▁
avg_min_cpu_MAPE,▁
avg_min_cpu_R2,▁


Loglama tamamlandı.
